In [418]:
import pandas as pd
import plotly.express as px

import warnings
warnings.filterwarnings('ignore')

from statsmodels.tsa.arima_model import ARIMA
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_squared_error

import plotly.graph_objects as go

import numpy as np


In [436]:
def to_int(value):
    try:
        n = value.strip()
        n = n.replace(",",'')
        return int(n)
    except:
        return value
    
def to_pct(value):
    try:
        n = value.replace("%",'')
        return float(n)
    except:
        return value
    
    
def to_date(value):
    return pd.to_datetime(value)

def predictions(column):
    column = column.dropna()
    data = column.values
    model = ARIMA(data, order=(1, 1, 1))
    model_fit = model.fit(disp=False)
    yhat = model_fit.predict(len(data),len(data)+3, typ='levels')
    return yhat

In [437]:
nat_header = ['MAP Region',
 'Year',
 'Quarter',
 'Properties',
 'Units',
 'Average Occupancy',
 'Median Occupancy',
 'Annual Absorption',
 'Annual Inventory Growth',
 'Properties Under Construction',
 'Units Under Construction',
 'Construction vs. Inventory',
 'Average Rent',
 'Annual Rent Growth',
 'Units Started',
 'Rolling 4-Quarter Units Started',
 'Rolling 4-Quarter Starts vs. Inventory']

In [438]:
df = pd.read_csv('data/national.csv', names = nat_header, header = 0)

In [439]:
drop = ['MAP Region','Year']
df = df.drop(drop,axis = 1)

In [440]:
df.Quarter = df.Quarter.apply(lambda x: to_date(x))
df.Properties = df.Properties.apply(lambda x:to_int(x))
df.Units = df.Units.apply(lambda x: to_int(x))
df['Average Occupancy'] = df['Average Occupancy'].apply(lambda x: to_pct(x))
df['Median Occupancy'] = df['Median Occupancy'].apply(lambda x: to_pct(x))
df['Annual Absorption'] = df['Annual Absorption'].apply(lambda x: to_pct(x))
df['Annual Inventory Growth'] = df['Annual Inventory Growth'].apply(lambda x: to_pct(x))
df['Properties Under Construction'] = df['Properties Under Construction'].apply(lambda x: to_int(x))
df['Units Under Construction'] = df['Units Under Construction'].apply(lambda x: to_int(x))
df['Construction vs. Inventory'] = df['Construction vs. Inventory'].apply(lambda x: to_pct(x))
df['Average Rent'] = df['Average Rent'].apply(lambda x: to_int(x))
df['Annual Rent Growth'] = df['Annual Rent Growth'].apply(lambda x: to_pct(x))
df['Units Started'] = df['Units Started'].apply(lambda x: to_int(x))
df['Rolling 4-Quarter Units Started'] = df['Rolling 4-Quarter Units Started'].apply(lambda x: to_int(x))
df['Rolling 4-Quarter Starts vs. Inventory'] = df['Rolling 4-Quarter Starts vs. Inventory'].apply(lambda x: to_pct(x))

## Add in Predictions

In [441]:
future = ['3Q2019','4Q2019','1Q2020','2Q2020']
future = [to_date(x) for x in future]

In [442]:
cols = df.columns.to_list()
cols = cols[1:]

In [443]:
df['Rolling 4-Quarter Units Started'][2]

nan

In [444]:
df

,Quarter,Properties,Units,Average Occupancy,Median Occupancy,Annual Absorption,Annual Inventory Growth,Properties Under Construction,Units Under Construction,Construction vs. Inventory,Average Rent,Annual Rent Growth,Units Started,Rolling 4-Quarter Units Started,Rolling 4-Quarter Starts vs. Inventory
0,2005-10-01,3672,459477,91.3,94.6,NaN,NaN,170,19259,4.2,2800,NaN,NaN,NaN,NaN
1,2006-01-01,3703,464518,91.0,94.6,NaN,NaN,163,17339,3.7,2836,NaN,2836.0,NaN,NaN
2,2006-04-01,3717,466392,91.4,94.8,NaN,NaN,160,17553,3.8,2865,NaN,2598.0,NaN,NaN
3,2006-07-01,3734,468860,91.3,94.6,NaN,NaN,180,19026,4.1,2878,NaN,4007.0,NaN,NaN
4,2006-10-01,3745,470864,91.3,94.8,2.5,2.5,192,21057,4.5,2908,3.6,4331.0,13772.0,2.9
5,2007-01-01,3769,474182,91.3,94.9,2.5,2.1,178,21234,4.5,2952,3.5,3846.0,14782.0,3.1
6,2007-04-01,3779,476834,90.9,94.3,1.8,2.2,180,21310,4.5,2977,3.8,3221.0,15405.0,3.2
7,2007-07-01,3790,478994,90.8,94.2,1.6,2.2,183,21583,4.5,2994,3.7,2689.0,14087.0,2.9
8,2007-10-01,3810,482533,90.5,93.8,1.5,2.5,193,22590,4.7,3019,3.8,5211.0,14967.0,3.1
9,2008-01-01,3835,487343,89.8,93.0,1.0,2.8,201,22668,4.7,3069,3.6,4903.0,16024.0,3.3


In [447]:
for each in cols:
    
    p = predictions(df[each])
    
    f = pd.DataFrame(future, columns = ['Quarter'])
    f['Predicted'] = p
    print(p)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x = df.Quarter, y = df[each],
                        mode = 'lines',
                        name = 'Historical'))
    
    fig.add_trace(go.Scatter(x = f.Quarter, y = f.Predicted,
                        mode = 'lines',
                        name = 'Predicted'))
    
    fig.update_layout(
        title = {
            'text': "{} vs Time".format(each),
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
        },
        xaxis_title = "Time",
        yaxis_title = each,
    )
    fig.show()
    fig.write_image("{} vs Time.png".format(each))

[5014.63306769 5047.94212541 5080.94032762 5113.64029472]


[637773.81948794 642066.58859602 646308.47379644 650502.45605768]


[88.23057486 88.09990962 88.05751286 87.99354783]


[91.96204383 91.88094986 91.84190886 91.79051297]


[3.00859731 3.01964109 3.02926485 3.03971284]


...

[4225.29391109 4250.69009971 4276.11257847 4301.54181504]


[2.60872579 2.55860387 2.52580095 2.50028678]


[4166.69990689 4220.58871483 4242.09308213 4262.50478443]


[16848.40055942 16120.63254649 15635.38615411 15315.43358306]


[2.62033656 2.49768657 2.41310612 2.35394583]
